## Packages

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
os.chdir('/Users/connorvoglewede/git-folder/boba')

In [195]:
import pandas as pd
import logging
import numpy as np
from bs4 import BeautifulSoup
import requests
import io
import re
from lxml import html 
from tqdm import tqdm
import json
import warnings 
import boba.scrapers as boba
import boba.preprocessing as pp


warnings.filterwarnings("ignore")


## Parameters

In [7]:
season = 2020

## Fangraphs

#### Hitters

In [50]:
# hitters_df = boba.fangraphs_h_season(season=2020)
# seasons = list(np.arange(1993,2015,1))
# path = 'data/raw/hitters/fangraphs/season/'
# for x in tqdm(seasons):
#     df = boba.fangraphs_h_season(season=x)
#     df.to_csv(path+str(x)+'.csv')

In [120]:
# test = pd.read_csv('../data/raw/hitters/fangraphs/season/2017.csv',index_col=0)
# test

#### Pitchers

In [119]:
# pitcher_df = boba.fangraphs_p_season(season=2020)
# seasons = list(np.arange(1993,2021,1))
# path = 'data/raw/pitchers/fangraphs/season/'
# for x in tqdm(seasons):
#     df = boba.fangraphs_p_season(season=x)
#     df.to_csv(path+str(x)+'.csv')

In [123]:
# test = pd.read_csv('../data/raw/pitchers/fangraphs/season/2017.csv',index_col=0)
# test

## Statcast

#### Hitters

In [105]:
# hitters_df = boba.statcast_h_season(2020)
# path = 'data/raw/hitters/statcast/season/'
# hitters_df.to_csv(path+str(2020)+'.csv')
# seasons = list(np.arange(2015,2021,1))
# for x in tqdm(seasons):
#     df = boba.statcast_h_season(season=x)
#     df.to_csv(path+str(x)+'.csv')


#### Pitchers

In [115]:
# pitcher_df = boba.statcast_p_season(2020)
# path = 'data/raw/pitchers/statcast/season/'
# pitcher_df.to_csv(path+str(2020)+'.csv')
# pitcher_df
# seasons = list(np.arange(2015,2021,1))
# for x in tqdm(seasons):
#     df = boba.statcast_p_season(season=x)
#     df.to_csv(path+str(x)+'.csv')

## Gather Sources

In [124]:
# fg_h = boba.gather_season_data(position_group='hitters',source = 'fangraphs',start_season=1993, end_season=2021)
# sc_h = boba.gather_season_data(position_group='hitters',source = 'statcast', start_season=2015, end_season=2021)

# fg_p = boba.gather_season_data(position_group='pitchers',source = 'fangraphs',start_season=1993, end_season=2021)
# sc_p = boba.gather_season_data(position_group='pitchers',source = 'statcast', start_season=2015, end_season=2021)

## Gather Projection System Historical Data

In [238]:
position_group='hitters'
start_season = 2019
end_season = 2021
systems = ['atc','zips','thebat','steamer']
# systems = ['steamer']
cols = ['Name','Team','playerid','PA','AVG','OBP','SLG','R','RBI','HR','SB','CS']

base_path = 'data/raw/'+position_group+'/projection_systems/'
seasons_list = list(np.arange(start_season,end_season+1,1))
data = pd.DataFrame(columns = cols) 
for sys in systems:
    for x in seasons_list:
        path = base_path+sys+'/'+str(x)+'.csv'
        try:
            df = pd.read_csv(path)
            df = df[cols]
            df['season'] = int(x)
            df['system'] = sys
            data = data.append(df)
        except:
            pass


stmr_hist_cols = ['MLBAM','PA','AVG','OBP','SLG']
stmr_hist = pd.DataFrame(columns = stmr_hist_cols)
for x in list(np.arange(2010,2012,1)):
    path = base_path+'steamer'+'/'+str(x)+'.csv'
    df = pd.read_csv(path)
    df = df[stmr_hist_cols]
    df['season'] = int(x)
    df['system'] = 'steamer'
    stmr_hist = stmr_hist.append(df)

for x in list(np.arange(2012,2019,1)):
    path = base_path+'steamer'+'/'+str(x)+'.csv'
    df = pd.read_csv(path)
    df = df[['mlbamid','PA','AVG','OBP','SLG']]
    df = df.rename(columns = {'mlbamid':'MLBAM'})
    df['season'] = int(x)
    df['system'] = 'steamer'
    stmr_hist = stmr_hist.append(df)
stmr_hist = stmr_hist[['system'] + ['season'] + stmr_hist_cols]

id_map = pp.load_ID_map(2021)


stmr_hist_id = pd.merge(stmr_hist,id_map,left_on = 'MLBAM',right_on = 'MLBID')
stmr_hist_id = stmr_hist_id.rename(columns = {'IDFANGRAPHS':'playerid'})
stmr_hist_id = stmr_hist_id[['system','season','playerid','PA','AVG','OBP','SLG']]

data = data.append(stmr_hist_id)
data = data[['system'] + ['season'] + cols] 
data['NetSB'] = data['SB']-data['CS']
data.to_csv(base_path+'master.csv')

In [234]:
position_group='pitchers'
start_season = 2019
end_season = 2021
systems = ['atc','zips','thebat','steamer']
# systems = ['steamer']
cols = ['Name','Team','playerid','IP','ERA']

base_path = 'data/raw/'+position_group+'/projection_systems/'
seasons_list = list(np.arange(start_season,end_season+1,1))
data = pd.DataFrame(columns = cols) 
for sys in systems:
    for x in seasons_list:
        path = base_path+sys+'/'+str(x)+'.csv'
        try:
            df = pd.read_csv(path)
            df = df[cols]
            df['season'] = int(x)
            df['system'] = sys
            data = data.append(df)
        except:
            pass


stmr_hist_cols = ['MLBAM','IP','ERA']
stmr_hist = pd.DataFrame(columns = stmr_hist_cols)
for x in list(np.arange(2010,2013,1)):
    path = base_path+'steamer'+'/'+str(x)+'.csv'
    df = pd.read_csv(path)
    df = df[stmr_hist_cols]
    df['season'] = int(x)
    df['system'] = 'steamer'
    stmr_hist = stmr_hist.append(df)

for x in list(np.arange(2013,2019,1)):
    path = base_path+'steamer'+'/'+str(x)+'.csv'
    df = pd.read_csv(path)
    df = df[['mlbamid','IP','ERA']]
    df = df.rename(columns = {'mlbamid':'MLBAM'})
    df['season'] = int(x)
    df['system'] = 'steamer'
    stmr_hist = stmr_hist.append(df)
stmr_hist = stmr_hist[['system'] + ['season'] + stmr_hist_cols]

id_map = pp.load_ID_map(2021)


stmr_hist_id = pd.merge(stmr_hist,id_map,left_on = 'MLBAM',right_on = 'MLBID')
stmr_hist_id = stmr_hist_id.rename(columns = {'IDFANGRAPHS':'playerid'})
stmr_hist_id = stmr_hist_id[['system','season','playerid','IP','ERA']]

data = data.append(stmr_hist_id)
data = data[['system'] + ['season'] + cols]     
data.to_csv(base_path+'master.csv')
